In [23]:
'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ~!@#$%^&*()_+-=[]{}\|;',./:\"<>? "

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [3]:
import numpy as np

def preprocess(filename):
    ret = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.lower()
            last_space = True
            new_line = []
            for i in line:
                if i.isalpha():
                    v = ord(i)-ord('a')+1
                    new_line += [[v]]
                    last_space = False
                else:
                    if last_space == False:
                        new_line += [[27]]
                        last_space = True
            ret += [new_line]
    return ret

jp_x = preprocess('jp.txt')
en_x = preprocess('en.txt')
x = jp_x + en_x
y = [[1.0,0.0] for i in jp_x] + [[0.0,1.0] for i in en_x]

In [4]:
print(len(x))
print(len(x[1]))

662
39


In [5]:
def align(data):
    max_len = 0
    for row in data:
        if len(row) > max_len:
            max_len = len(row)
    ret = []
    for row in data:
        ret += [row + [[0]]*(max_len-len(row))]
    return ret, max_len

x, seq_max_len = align(x)

In [6]:
print(len(x[0]))
print(seq_max_len)

75
75


In [7]:
import random
r_index = list(range(len(x)))
random.shuffle(r_index)
train_x = [x[i] for i in r_index[:int(len(r_index)*0.8)]]
train_y = [y[i] for i in r_index[:int(len(r_index)*0.8)]]
test_x = [x[i] for i in r_index[int(len(r_index)*0.8):]]
test_y = [y[i] for i in r_index[int(len(r_index)*0.8):]]

In [8]:
print(train_x[:2])
print(len(train_x))
print(len(train_y))
print(seq_max_len)
print(train_y[:10])

[[[25], [5], [1], [8], [27], [8], [1], [27], [8], [1], [27], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]], [[25], [15], [21], [27], [19], [5], [5], [27], [23], [9], [20], [8], [27], [25], [15], [21], [18], [27], [5], [25], [5], [19], [27], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]]
529
529
75
[[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0]]


In [9]:
import tensorflow as tf
print(tf.__version__)

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


1.4.0


In [20]:
n_hidden = 50
num_steps = 5000
n_classes = 2

def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

graph = tf.Graph()
with graph.as_default(), tf.device('/cpu:0'):
    
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    
    def dynamicRNN(x, weights, biases):
        seqlen = length(x)
        x = tf.unstack(x, seq_max_len, 1)
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)
        outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                    sequence_length=seqlen)

        # When performing dynamic calculation, we must retrieve the last
        # dynamically computed output, i.e., if a sequence length is 10, we need
        # to retrieve the 10th output.
        # However TensorFlow doesn't support advanced indexing yet, so we build
        # a custom op that for each sample in batch size, get its length and
        # get the corresponding relevant output.

        # 'outputs' is a list of output at every timestep, we pack them in a Tensor
        # and change back dimension to [batch_size, n_step, n_input]
        outputs = tf.stack(outputs)
        outputs = tf.transpose(outputs, [1, 0, 2])

        # Hack to build the indexing and retrieve the right output.
        batch_size = tf.shape(outputs)[0]
        # Start indices for each sample
        index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
        # Indexing
        outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

        # Linear activation, using outputs computed above
        return tf.matmul(outputs, weights['out']) + biases['out']
        
    x = tf.placeholder("float", [None, seq_max_len,1])
    y = tf.placeholder("float", [None, n_classes])
    inputs = x
    labels = y
    logits = dynamicRNN(x, weights, biases)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(loss)
    
    correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        feed_dict = {inputs: train_x, labels: train_y}
        _, l, predictions, acc = session.run([optimizer, loss, logits, accuracy],feed_dict = feed_dict)
        if (step % 100 == 0):
            print('Loss at step %d: %f acc: %f' % (step, l,acc))
    print("Testing Accuracy:", session.run(accuracy, feed_dict={inputs: test_x, labels: test_y}))

/home/ziyi/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss at step 0: 2.806529 acc: 0.504726
Loss at step 100: 0.688050 acc: 0.508507
Loss at step 200: 0.678933 acc: 0.546314
Loss at step 300: 0.672208 acc: 0.578450
Loss at step 400: 0.666399 acc: 0.584121
Loss at step 500: 0.660915 acc: 0.612476
Loss at step 600: 0.655405 acc: 0.603025
Loss at step 700: 0.649568 acc: 0.629490
Loss at step 800: 0.643081 acc: 0.623819
Loss at step 900: 0.635670 acc: 0.638941
Loss at step 1000: 0.626802 acc: 0.637051
Loss at step 1100: 0.613682 acc: 0.661626
Loss at step 1200: 0.575704 acc: 0.701323
Loss at step 1300: 0.430961 acc: 0.814745
Loss at step 1400: 0.381359 acc: 0.841210
Loss at step 1500: 0.386692 acc: 0.846881
Loss at step 1600: 0.351815 acc: 0.854442
Loss at step 1700: 0.340029 acc: 0.856333
Loss at step 1800: 0.330225 acc: 0.865785
Loss at step 1900: 0.320089 acc: 0.879017
Loss at step 2000: 0.316310 acc: 0.877127
Loss at step 2100: 0.309775 acc: 0.877127
Lo

In [19]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        feed_dict = {inputs: train_x, labels: train_y}
        _, l, predictions, acc = session.run([optimizer, loss, logits, accuracy],feed_dict = feed_dict)
        if (step % 100 == 0):
            print('Loss at step %d: %f acc: %f' % (step, l,acc))
    print("Testing Accuracy:", session.run(accuracy, feed_dict={inputs: test_x, labels: test_y}))

Initialized
Loss at step 0: 0.733895 acc: 0.504726
Loss at step 100: 0.730986 acc: 0.504726
Loss at step 200: 0.728342 acc: 0.504726
Loss at step 300: 0.725835 acc: 0.504726
Loss at step 400: 0.723452 acc: 0.504726


KeyboardInterrupt: 